## DNN
### with pretraining, non-freeze previous layers
Sep 25, 2018

e04_nf-> batch size 256, 1/10 train dev data, each layer 1024 cell

e04v2_nf-> batch size 256, 1/10 train dev data, each layer 2048 cell

### imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys

import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

import keras.backend as K

import matplotlib.pyplot as plt
import pixiedust

Using TensorFlow backend.


Pixiedust database opened successfully


### Variables
- Read parameters from .mat files
- save in Opt object

In [2]:
# %%pixie_debugger
Data_VERSION = '_e10v5'
Test_Data_VERSION = '_e10v5'

Code_VERSION = '_e04v2_nf'

In [3]:
DNN_DATA_FILE = "./dnn_models/DNN_datas"+Data_VERSION+".mat"

DNN_TEST_FILE = "./dnn_models/Test_datas"+Test_Data_VERSION+".mat"


DNN_PARAMS_FILE = "./dnn_models/DNN_params"+Data_VERSION+".mat"

DNN_NET_FILE = "./dnn_models/DNN_net"+Code_VERSION+".mat"


# Best model, after adding 1st dense layer
MODEL_FILE_1 = "./dnn_models/weights1"+ Code_VERSION+"_{epoch:02d}.h5"

# Best model, after adding 2nd dense layer
MODEL_FILE_2 = "./dnn_models/weights2"+ Code_VERSION+"_{epoch:02d}.h5"

# Best model, after adding 3rd dense layer (final)
MODEL_FILE_3 = "./dnn_models/weights3"+ Code_VERSION+"_{epoch:02d}.h5"


#dummy save file
SAVE_MODEL_FILE = "./dnn_models/py_model"+ Code_VERSION+".h5"

# estimated real+imag for test dataset
OUTPUT_FILE = "./dnn_models/Real_Imag"+Code_VERSION+".mat"

### Object class

In [4]:
class Opts:

    def __init__(self, FILE_PARA, FILE_DATA, FILE_TEST):
        
        # Basic parameters
        with h5py.File(FILE_PARA, 'r') as f:
            key_list = list(f.keys())
            print('File name <{0}>\nOpt keys (Total {1}):'.format(FILE_PARA,len(f['opts'].items())) )

            for k, v in f['opts'].items():

                print(k,end=', ')

                if k == 'ARMA_order':
                    self.ARMA_order = int(np.array(v)[0][0])
                    
                elif k == 'ada_grad_eps':
                    self.ada_grad_eps = np.array(v)[0][0]
                    
                elif k == 'ada_sgd_scale':
                    self.ada_sgd_scale = np.array(v)[0][0]
                    
                elif k == 'change_momentum_point':
                    self.change_momentum_point = int(np.array(v)[0][0])
                    
                elif k == 'cost_function':
                    self.cost_function = ""
                    for c in np.array(v):
                        self.cost_function += chr(c[0])

                elif k == 'cv_interval':
                    self.cv_interval = int(np.array(v)[0][0])
                    
                elif k == 'dim_input':
                    self.dim_input = int(np.array(v)[0][0])
                    
                elif k == 'dim_output':
                    self.dim_output = int(np.array(v)[0][0])
                    
                elif k == 'drop_ratio':
                    self.drop_ratio = np.array(v)[0][0]
                    
                elif k == 'eval_on_gpu':
                    self.eval_on_gpu = int(np.array(v)[0][0])
                    
                elif k == 'final_momentum':
                    self.final_momentum = int(np.array(v)[0][0])
                   
                elif k == 'hid_struct':
                    self.hid_struct = np.array(v)
                    
                elif k == 'initial_momentum':
                    self.initial_momentum = np.array(v)[0][0]
                    
                elif k == 'isDropout':
                    self.isDropout = 0
                    
                elif k == 'isDropoutInput':
                    self.isDropoutInput = int(np.array(v)[0][0])
                    
                elif k == 'isGPU':
                    self.isGPU = int(np.array(v)[0][0])
                    
                elif k == 'isNormalize':
                    self.isNormalize = int(np.array(v)[0][0])
                    
                elif k == 'isPretrain':
                    self.isPretrain = int(np.array(v)[0][0])
                    
                elif k == 'learner':
                    self.learner = ""
                    for c in np.array(v):
                        self.learner += chr(c[0])

                elif k == 'net_struct':
                    self.net_struct = np.array(v)
#                     for n_s in np.array(v):
#                         print('Opts Net Stuct:',n_s[0])

                elif k == 'rbm_batch_size':
                    self.rbm_batch_size = int(np.array(v)[0][0])

                elif k == 'rbm_learn_rate_binary':
                    self.rbm_learn_rate_binary = np.array(v)

                elif k == 'rbm_learn_rate_real':
                    self.rbm_learn_rate_real = int(np.array(v)[0][0])

                elif k == 'rbm_max_epoch':
                    self.rbm_max_epoch = int(np.array(v)[0][0])

                elif k == 'save_on_fly':
                    self.save_on_fly = int(np.array(v)[0][0])

                elif k == 'sgd_batch_size':
                    self.sgd_batch_size = int(np.array(v)[0][0]) # BATCH_SIZE for training net
                    print("self.sgd_batch_size:",self.sgd_batch_size)

                elif k == 'sgd_learn_rate':
                    self.sgd_learn_rate = np.array(v)

                elif k == 'sgd_max_epoch':
                    self.sgd_max_epoch = int(np.array(v)[0][0])

                elif k == 'split_tanh1_c1':
                    self.split_tanh1_c1 = int(np.array(v)[0][0])

                elif k == 'split_tanh1_c2':
                    self.split_tanh1_c2 = int(np.array(v)[0][0])

                elif k == 'unit_type_hidden':
                    self.unit_type_hidden = ""
                    for c in np.array(v):
                        self.unit_type_hidden += chr(c[0])

                elif k == 'unit_type_output':
                    self.unit_type_output = ""
                    for c in np.array(v):
                        self.unit_type_output += chr(c[0])

        # Training and Dev Data 
        with h5py.File(FILE_DATA, 'r') as f:
            
            print('\n\nFile name <{0}>\nOpt h5py keys (Total {1}):'.format(FILE_DATA,len(f.keys())) )

            for k, v in f.items():
                print(k, end=', ')
                
                # Train Data
                if k == 'trData':
                    self.trData = np.transpose(np.array(v))
                    print("trData.shape: ", self.trData.shape)
                    print("trData-> mean:", np.mean(self.trData), ", var:", np.var(self.trData), ", std:",
                          np.std(self.trData), ", range:", (np.amin(self.trData),np.amax(self.trData)))
                elif k == 'trLabel_i':
                    self.trLabel_i = np.transpose(np.array(v))
                    print("trLabel_i.shape: ", self.trLabel_i.shape)
                    print("trLabel_i-> mean:", np.mean(self.trLabel_i), ", var:", np.var(self.trLabel_i), ", std:",
                          np.std(self.trLabel_i), ", range:", (np.amin(self.trLabel_i),np.amax(self.trLabel_i)))
                elif k == 'trLabel_r':
                    self.trLabel_r = np.transpose(np.array(v))
                    print("trLabel_r.shape: ", self.trLabel_r.shape)
                    print("trLabel_r-> mean:", np.mean(self.trLabel_r), ", var:", np.var(self.trLabel_r), ", std:",
                          np.std(self.trLabel_r), ", range:", (np.amin(self.trLabel_r),np.amax(self.trLabel_r)))
                elif k == 'trNumframes':
                    self.trNumframes = np.transpose(np.array(v))
                    print("trNumframes.shape: ", self.trNumframes.shape)
                    
                    
                # Dev Data
                elif k == 'cvData':
                    self.cvData = np.transpose(np.array(v))
                    print("cvData.shape: ", self.cvData.shape)
                    print("cvData-> mean:", np.mean(self.cvData), ", var:", np.var(self.cvData), ", std:",
                          np.std(self.cvData), ", range:", (np.amin(self.cvData),np.amax(self.cvData)))
                elif k == 'cvLabel_i':
                    self.cvLabel_i = np.transpose(np.array(v))
                    print("cvLabel_i.shape: ", self.cvLabel_i.shape)
                    print("cvLabel_i-> mean:", np.mean(self.cvLabel_i), ", var:", np.var(self.cvLabel_i), ", std:",
                          np.std(self.cvLabel_i), ", range:", (np.amin(self.cvLabel_i),np.amax(self.cvLabel_i)))
                elif k == 'cvLabel_r':
                    self.cvLabel_r = np.transpose(np.array(v))
                    print("cvLabel_r.shape: ", self.cvLabel_r.shape)
                    print("cvLabel_r-> mean:", np.mean(self.cvLabel_r), ", var:", np.var(self.cvLabel_r), ", std:",
                          np.std(self.cvLabel_r), ", range:", (np.amin(self.cvLabel_r),np.amax(self.cvLabel_r)))
                elif k == 'cvNumframes':
                    self.cvNumframes = np.transpose(np.array(v))
                    print("cvNumframes.shape: ", self.cvNumframes.shape)
                    

            self.trLabel = np.concatenate((self.trLabel_r, self.trLabel_i), axis=1)
            self.cvLabel = np.concatenate((self.cvLabel_r, self.cvLabel_i), axis=1)
 
            
        # Test Data 
        print(FILE_TEST, os.path.isfile(FILE_TEST))
        
        with h5py.File(FILE_TEST, 'r') as f:
            print('\n\nFile name <{0}>\nOpt h5py keys (Total {1}):'.format(FILE_TEST,len(f.keys())) )

            
            
            for k, v in f.items():
                print(k, end=', ')
                
                #Test Data
                if k == 'teData':
                    self.teData = np.transpose(np.array(v))
                    print("teData.shape: ", self.teData.shape)
                    print("teData-> mean:", np.mean(self.teData), ", var:", np.var(self.teData), ", std:",
                          np.std(self.teData), ", range:", (np.amin(self.teData),np.amax(self.teData)))
                    
                elif k == 'teLabel_i':
                    self.teLabel_i = np.transpose(np.array(v))
                    print("teLabel_i.shape: ", self.teLabel_i.shape)
                    print("teLabel_i-> mean:", np.mean(self.teLabel_i), ", var:", np.var(self.teLabel_i), ", std:",
                          np.std(self.teLabel_i), ", range:", (np.amin(self.teLabel_i),np.amax(self.teLabel_i)))
                    
                elif k == 'teLabel_r':
                    self.teLabel_r = np.transpose(np.array(v))
                    print("teLabel_r.shape: ", self.teLabel_r.shape)
                    print("teLabel_r-> mean:", np.mean(self.teLabel_r), ", var:", np.var(self.teLabel_r), ", std:",
                          np.std(self.teLabel_r), ", range:", (np.amin(self.teLabel_r),np.amax(self.teLabel_r)))
                
                elif k == 'teNumframes':
                    self.teNumframes = np.transpose(np.array(v))
                    print("teNumframes.shape: ", self.teNumframes.shape)
                    
            
            self.teLabel = np.concatenate((self.teLabel_r, self.teLabel_i), axis=1)


    def ready_batchID(self, total_num_samples, batch_size):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = []
        num_batch = math.ceil(total_num_samples/batch_size)

        for b in range( int(num_batch) ):
            s = b*batch_size
            e = (b+1)*batch_size -1

            if e >= total_num_samples:
                e = total_num_samples - 1

            batchID.append((s,e))

        return np.array(batchID,ndmin=2)


    def suffle_data(self, total_num_samples):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        return  np.random.permutation(total_num_samples)


    def next_batch(self, total_num_samples, batch_size, cycle):
        '''
        Currently we use this one. 
        
        parameters: 
            total_num_samples <int> = Shape[1] of data
            batch_size <int>        
            cycle <string>  = train, dev, test
        '''
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        while True:
            batchID = self.ready_batchID(total_num_samples, batch_size) 
            seq = self.suffle_data(total_num_samples)

            for batch in range(batchID.shape[0]):
                if cycle.lower()=='train':
                    x = opts.trData[ seq[batchID[batch][0]:batchID[batch][1] ] ]
                    y = opts.trLabel[ seq[batchID[batch][0]:batchID[batch][1] ] ]

                elif cycle.lower()=='dev':
                    x = opts.cvData[seq[batchID[batch][0]:batchID[batch][1]]]
                    y = opts.cvLabel[seq[batchID[batch][0]:batchID[batch][1]]]
                    
                elif cycle.lower()=='test':
                    x = opts.teData[seq[batchID[batch][0]:batchID[batch][1]]]
                    y = opts.teLabel[seq[batchID[batch][0]:batchID[batch][1]]]

                # print('Next Batch', x.shape, y.shape)
                yield [x, y]


In [5]:
# %%pixie_debugger
opts = Opts(DNN_PARAMS_FILE, DNN_DATA_FILE, DNN_TEST_FILE)

File name <./dnn_models/DNN_params_e10v5.mat>
Opt keys (Total 34):
ARMA_order, ada_grad_eps, ada_sgd_scale, change_momentum_point, cost_function, cv_interval, dim_input, dim_output, drop_ratio, eval_on_gpu, final_momentum, hid_struct, initial_momentum, isDropout, isDropoutInput, isGPU, isNormalize, isPretrain, learner, net_struct, rbm_batch_size, rbm_learn_rate_binary, rbm_learn_rate_real, rbm_max_epoch, save_on_fly, sgd_batch_size, self.sgd_batch_size: 1024
sgd_learn_rate, sgd_max_epoch, split_tanh1_c1, split_tanh1_c2, tr_mu, tr_std, unit_type_hidden, unit_type_output, 

File name <./dnn_models/DNN_datas_e10v5.mat>
Opt h5py keys (Total 8):
cvData, cvData.shape:  (449610, 1230)
cvData-> mean: -3.12831e-06 , var: 0.607645 , std: 0.779516 , range: (-9.6298428, 10.49406)
cvLabel_i, cvLabel_i.shape:  (449610, 963)
cvLabel_i-> mean: -1.2493e-05 , var: 0.0129563 , std: 0.113826 , range: (-10.0, 10.0)
cvLabel_r, cvLabel_r.shape:  (449610, 963)
cvLabel_r-> mean: 0.028573 , var: 0.0212577 , std

### Data/Label Visualization

### Old cost functions

### Custom loss function

In [6]:
def customLoss(yTrue,yPred):
    
    # Define loss and optimizer
    r,c = K.shape(yTrue)[0], K.shape(yTrue)[1] # opts.sgd_batch_size
    
    r= K.cast(r,dtype='float32') # because by default, K.shape tensors are dtype int32

    cost_r = K.sum(K.square(yTrue[:, :(c//2)]- yPred[:, :(c//2)]))
    cost_i = K.sum(K.square(yTrue[:, (c//2):]- yPred[:, (c//2):]))
    
    cost = 0.5*(cost_r+cost_i)/r
    return cost

### DNN variables

In [11]:
batch_size = 256 #opts.sgd_batch_size
epochs = 80

'''
n_train, n_train_dim = <number of samples for training, input dimentions>, final will be (1951920, 1230)
n_dev, n_dev_dim = <number of samples for development, input dimentions>, final will be (449610, 1230)
n_test, n_test_dim =<number of samples for testing, input dimentions>, final will be (72440, 1230)

'''
n_train, n_train_dim = (1951920, 1230) # opts.trData.shape = (1951920, 1230)
n_dev, n_dev_dim = (449610, 1230) # opts.cvData.shape =(449610, 1230)
n_test, n_test_dim = (72440, 1230) # opts.teData.shape =(72440, 1230)

n_hidden_1 = 1024  # 1st layer number of neurons
n_hidden_2 = 1024  # 2nd layer number of neurons
n_hidden_3 = 1024  # 3rd layer number of neurons

n_classes = (963 + 963)


### Pre-trained DNN

In [12]:
model = Sequential()

model.add(Dense(n_hidden_1, activation='relu', input_shape=(n_train_dim,),name='dense_1_e4v1'))
# we will add 2 more layers
model.add(Dense(n_classes, activation='linear'))


# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE_1, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(n_train, batch_size, cycle='train'),  
                    validation_data=opts.next_batch(n_dev, batch_size, cycle='dev'),
                    epochs=epochs, steps_per_epoch=n_train//batch_size,
                    validation_steps=n_dev//batch_size, 
                    verbose=1, callbacks=callbacks)
# history = model.fit_generator(opts.next_batch(n_train, batch_size, cycle='train'),  
#                     validation_data=opts.next_batch(n_dev, batch_size, cycle='dev'),
#                     epochs=epochs, steps_per_epoch=int(math.ceil(n_train/batch_size)),
#                     validation_steps=int(math.ceil(n_out_sz/batch_size)), 
#                     verbose=1, callbacks=callbacks)

model.save(SAVE_MODEL_FILE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1_e4v1 (Dense)         (None, 1024)              1260544   
_________________________________________________________________
dense_2 (Dense)              (None, 1926)              1974150   
Total params: 3,234,694
Trainable params: 3,234,694
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/80
7624/7624 [==============================] - 377s - loss: 16.0784 - acc: 0.0276 - mean_squared_error: 0.0167 - mean_absolute_error: 0.0455 - mean_absolute_percentage_error: 24548.6627 - cosine_proximity: -1.1815e-04 - val_loss: 15.5544 - val_acc: 0.0274 - val_mean_squared_error: 0.0162 - val_mean_absolute_error: 0.0444 - val_mean_absolute_percentage_error: 7840.6323 - val_cosine_proximity: -1.1629e-04
Epoch 2/80
7624/7624 [==============================] - 357s - loss: 15.7809 - acc: 0.0295 - mean_squared_e

In [13]:
Best_weight_1="./dnn_models/weights1"+ Code_VERSION+"_02.h5"

### Add 2nd hidden layer

In [14]:
# SAVE_MODEL_FILE_2 = "./dnn_models/py_model_2_"+ Code_VERSION+".h5"
model = load_model(Best_weight_1, custom_objects={'customLoss':customLoss})

model.pop()

model.add(Dense(n_hidden_2, activation='relu',name='dense_2_e4v1'))
# we will add 1 more layers
model.add(Dense(n_classes, activation='linear'))



# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE_2, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(n_train, batch_size, cycle='train'),  
                    validation_data=opts.next_batch(n_dev, batch_size, cycle='dev'),
                    epochs=epochs, steps_per_epoch=n_train//batch_size,
                    validation_steps=n_dev//batch_size, 
                    verbose=1, callbacks=callbacks)

# history = model.fit_generator(opts.next_batch(opts.trData.shape[0], batch_size),  
#                     validation_data=opts.next_batch(opts.cvData.shape[0], batch_size, isTrainCycle=False),
#                     epochs=epochs, steps_per_epoch=int(math.ceil(n_input_sz/batch_size)),
#                     validation_steps=int(math.ceil(n_out_sz/batch_size)), 
#                     verbose=1, callbacks=callbacks)

model.save(SAVE_MODEL_FILE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1_e4v1 (Dense)         (None, 1024)              1260544   
_________________________________________________________________
dense_2_e4v1 (Dense)         (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1926)              1974150   
Total params: 4,284,294
Trainable params: 4,284,294
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/80
7624/7624 [==============================] - 492s - loss: 15.6783 - acc: 0.0259 - mean_squared_error: 0.0163 - mean_absolute_error: 0.0426 - mean_absolute_percentage_error: 4769.8290 - cosine_proximity: -1.2926e-04 - val_loss: 15.3804 - val_acc: 0.0252 - val_mean_squared_error: 0.0160 - val_mean_absolute_error: 0.0420 - val_mean_absolute_percentage_error: 2651.7655 - val_cosine

In [15]:
Best_weight_2="./dnn_models/weights2"+ Code_VERSION+"_04.h5"

### Add 3rd hidden layer

In [16]:
# SAVE_MODEL_FILE_3 = "./dnn_models/py_model_3_"+ Code_VERSION+".h5"
model = load_model(Best_weight_2, custom_objects={'customLoss':customLoss})

model.pop()

model.add(Dense(n_hidden_2, activation='relu',name='dense_3_e4v1'))
model.add(Dense(n_classes, activation='linear'))


# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE_3, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(n_train, batch_size, cycle='train'),  
                    validation_data=opts.next_batch(n_dev, batch_size, cycle='dev'),
                    epochs=epochs, steps_per_epoch=n_train//batch_size,
                    validation_steps=n_dev//batch_size, 
                    verbose=1, callbacks=callbacks)

# history = model.fit_generator(opts.next_batch(opts.trData.shape[0], batch_size),  
#                     validation_data=opts.next_batch(opts.cvData.shape[0], batch_size, isTrainCycle=False),
#                     epochs=epochs, steps_per_epoch=int(math.ceil(n_input_sz/batch_size)),
#                     validation_steps=int(math.ceil(n_out_sz/batch_size)), 
#                     verbose=1, callbacks=callbacks)


model.save(SAVE_MODEL_FILE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1_e4v1 (Dense)         (None, 1024)              1260544   
_________________________________________________________________
dense_2_e4v1 (Dense)         (None, 1024)              1049600   
_________________________________________________________________
dense_3_e4v1 (Dense)         (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1926)              1974150   
Total params: 5,333,894
Trainable params: 5,333,894
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/80
7624/7624 [==============================] - 428s - loss: 15.6238 - acc: 0.0260 - mean_squared_error: 0.0162 - mean_absolute_error: 0.0421 - mean_absolute_percentage_error: 3296.6255 - cosine_proximity: -1.3270e-04 - val_loss: 15.3296 - val_acc: 0.

In [17]:
for layer in model.layers:
    g=layer.get_config()
    print (g,end='\n\n')

{'name': 'dense_1_e4v1', 'trainable': True, 'batch_input_shape': (None, 1230), 'dtype': 'float32', 'units': 1024, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}

{'name': 'dense_2_e4v1', 'trainable': True, 'units': 1024, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}

{'name': 'dense_3_e4v1', 'trainable': True, 'units': 1024, 'activatio

In [18]:
Best_weight_3="./dnn_models/weights3"+ Code_VERSION+"_04.h5"

### Predict

In [19]:
model = load_model(Best_weight_3, custom_objects={'customLoss':customLoss})


y_hat = model.predict(opts.teData, verbose=1)

print(y_hat.shape)

72160/72440 [============================>.] - ETA: 0s(72440, 1926)


In [20]:
sio.savemat(OUTPUT_FILE, {'y_hat':y_hat})

### Unwrapping

In [ ]:
def unwrapAugmentedTF_wAvg(impt_mag,num_per_side,unwrap_mag=None):
    
#     Description: Unwrap the augmented time-frequency (T-F) representation and compute the average.

#     Input:
#         impt_mag: wrapped T-F representation with dimensions (2*T+1)*d x m
#         T: number of frames to left and right of each frame used to augmented T-F representation
#     Output:
#         unwrap_avgmag: unwrapped and averaged T-F representation with dimensions d x m
#         unwrap_mag: unwrapped T-F representation with dimensions d x (2*T+1) x m
#
        
        sliding_window_len       = 2*num_per_side + 1
        (numWrapFreqs,numFrames) = impt_mag.shape
        numFreqs                 = numWrapFreqs//sliding_window_len
        unwrap_avgmag            = np.zeros((numFreqs,numFrames))
        
        if num_per_side > 0:
            if unwrap_mag is None:
            
                unwrap_mag        = np.zeros((numFreqs,sliding_window_len+1,numFrames))
                curr_ind_location = np.ones((numFrames,1),dtype=int)
                

                for frameNum in range(numFrames):
                    
                    # Get the indices for the frames used in this augmented matrix
                    frame_inds=[]
                    for inds in range(frameNum-num_per_side,frameNum+num_per_side+1):
                        if inds<0:
                            frame_inds.append(0)
                        elif inds>=numFrames:
                            frame_inds.append(numFrames-1)
                        else:
                            frame_inds.append(inds)
                    
                    # Unwrap the data for this frame
                    slid_win_data = np.reshape( impt_mag[:,frameNum], (numFreqs,sliding_window_len)) #Size d x (2*T + 1)
                    
                    for ind_num in range(len(frame_inds)):

                        slid = np.array(slid_win_data[:,ind_num], ndmin=2).T
                        unwrap_mag[:,curr_ind_location[frame_inds[ind_num]], frame_inds[ind_num]] = slid 
                        
                        # Update counters
                        curr_ind_location[frame_inds[ind_num]] = curr_ind_location[frame_inds[ind_num]] + 1


            temp = np.zeros((numFreqs,sliding_window_len+1))
            
            for frameNum in range(numFrames):
#                 print(unwrap_mag.shape)
                
                unwrap_avgmag[:,frameNum] = np.mean(unwrap_mag[:,:,frameNum],axis=1)
#                 print(unwrap_mag.shape)
#                 return

            
        else:
            unwrap_avgmag = impt_mag;
            unwrap_mag = 0;
            
        return unwrap_avgmag,unwrap_mag